<a href="https://colab.research.google.com/github/OlyKoek/MyStudy-AI-app/blob/feature-update-ml/create_multimodal_embeding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# マルチモーダルなエンベディングモデルの作成
# Mini-CLIP: Text + Image shared embedding

- Text: multilingual MiniLM (TinyBERT)
- Image: MobileNetV3-Small
- Projection Head: 256 dimention shared space
- Loss: CLIP-style contrastive loss
- VectorStore: SimpleVetorDB    

# Import and Config

In [1]:
!pip install torch torchvision torchaudio --quiet
!pip install transformers umap-learn matplotlib pandas scikit-learn plotly --quiet

!pip install kaggle

import os
import json
from io import BytesIO

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import AdamW

from transformers import AutoTokenizer, AutoModel

import torchvision.models as models
import torchvision.transforms as T

import numpy as np
import pandas as pd
import umap
import matplotlib.pyplot as plt
import plotly.express as px

from PIL import Image
import requests
from IPython.display import display

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("DEVICE:", DEVICE)

# モデルと次元数
TEXT_MODEL_NAME = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
EMBED_DIM = 256

DEVICE: cpu


In [2]:
from google.colab import userdata

os.environ["KAGGLE_USERNAME"] = userdata.get('username')
os.environ["KAGGLE_KEY"] = userdata.get('token')

!kaggle datasets list

ref                                                             title                                                     size  lastUpdated                 downloadCount  voteCount  usabilityRating  
--------------------------------------------------------------  --------------------------------------------------  ----------  --------------------------  -------------  ---------  ---------------  
wardabilal/spotify-global-music-dataset-20092025                Spotify Global Music Dataset (2009–2025)               1289021  2025-11-11 09:43:05.933000           3288         58  1.0              
sadiajavedd/students-academic-performance-dataset               Students_Academic_Performance_Dataset                     8907  2025-10-23 04:16:35.563000           9208        230  1.0              
zahranusrat/salary                                              Salary Insights by Job Role (2024)                      110295  2025-11-11 07:02:27.350000           1309         24  1.0              


# Utils

In [3]:
def cosine_sim(a: np.ndarray, b: np.ndarray) -> float:
    """
    ベクトル a, b のコサイン類似度を返すための関数
    """
    a_flat = a.flatten()
    b_flat = b.flatten()
    denom = (np.linalg.norm(a_flat) * np.linalg.norm(b_flat) + 1e-8)
    return float(np.dot(a_flat, b_flat) / denom)


def load_image(x):
    """
    Imageを読み込むための関数
    入力 x:
      - str(URL or file path)
      - PIL.Image.Image
    """
    if isinstance(x, Image.Image):
        return x.convert("RGB")

    if isinstance(x, str):
        if x.startswith("https://") or x.startswith("http://"):
            response = requests.get(x)
            img = Image.open(BytesIO(response.content))
            return img.convert("RGB")
        else:
            img = Image.open(x)
            return img.convert("RGB")

    raise ValueError(f"Unsupported image input type: {type(x)}")


def plot_umap_matplotlib(embs, labels, texts=None, title="UMAP"):
    """
    MatplotlibでUMAPを描画
    embs: (N, D) numpy array
    labels: list[str] same length as N
    texts:  list[str] hover表示用（任意）
    """
    reducer = umap.UMAP(n_neighbors=10, min_dist=0.1, metric="cosine")
    coords = reducer.fit_transform(embs)

    plt.figure(figsize=(8, 6))
    for i, lab in enumerate(labels):
        plt.scatter(coords[i, 0], coords[i, 1])
        if texts is not None:
            plt.text(coords[i, 0], coords[i, 1], texts[i][:10], fontsize=8)
    plt.title(title)
    plt.show()


def plot_umap_plotly(embs, labels, texts=None, extra_meta=None, title="UMAP (Interactive)"):
    """
    PlotlyでインタラクティブなUMAPを描画
    """
    reducer = umap.UMAP(n_neighbors=10, min_dist=0.1, metric="cosine")
    coords = reducer.fit_transform(embs)

    data = {
        "x": coords[:, 0],
        "y": coords[:, 1],
        "type": labels,
    }
    if texts is not None:
        data["text"] = texts
    if extra_meta is not None:
        for k, v in extra_meta.items():
            data[k] = v

    df = pd.DataFrame(data)
    fig = px.scatter(
        df,
        x="x",
        y="y",
        color="type",
        hover_data=list(data.keys()),
        title=title,
        width=800,
        height=600,
    )
    fig.show()


def unpickle(file):
    """
    CIFARフォーマットのpickleを読み込む
    """
    with open(file, 'rb') as fo:
        d = pickle

## Core Classes
- TextEncoder
- ImageEncoder
- MiniCLIP
- SimpleVectorDB

In [4]:
class TextEncoder(nn.Module):
    """
    TinyBERT系でテキストをエンコードするクラス
    Linear Projectionで256次元に射影変換
    """
    def __init__(self, model_name: str, out_dim: int = EMBED_DIM, device: torch.device = DEVICE):
        super().__init__()
        self.device = device
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModel.from_pretrained(model_name).to(device)
        hidden = self.model.config.hidden_size
        self.projector  = nn.Linear(hidden, out_dim).to(device)

    def forward(self, inputs):
        """
        HuggingFaceスタイルのForward関数を参考に実装
        B: バッチサイズ
        3: チャネル数(RGB)
        H, W: 画像の高さと幅
        T: トークン数
        H: モデルの隠れ層次元数
        out_dim: 射影後の次元数
        """
        outputs = self.model(**inputs)
        embeddings = outputs.last_hidden_state.mean(dim=1)  # (B, T, H) -> (B, H)
        projected = self.projector(embeddings)  # (B, out_dim)
        return projected

    def encode(self, texts, normalize: bool = True) -> torch.Tensor:
        """
        textsをエンコードしてベクトルを返す関数

        texts: list[str]
        normalize: bool - 出力ベクトルを正規化するかどうか
        """
        if isinstance(texts, str):
            texts = [texts]

        inputs = self.tokenizer(
            texts,
            padding=True,
            truncation=True,
            return_tensors="pt",
        ).to(self.device)

        with torch.no_grad():
            outputs = self.model(**inputs)
            emb = outputs.last_hidden_state.mean(dim=1)  # (B, H)

        proj = self.projector(emb)  # (B, out_dim)

        if normalize:
            proj = proj / (proj.norm(dim=1, keepdim=True) + 1e-8)

        return proj

In [5]:
class ImageEncoder(nn.Module):
    """
    MobileNetV3-Smallで画像をエンコードするクラス
    Linear Projectionで256次元に射影変換
    """
    def __init__(self, out_dim: int = EMBED_DIM, device: torch.device = DEVICE):
        super().__init__()
        self.device = device
        self.model = models.mobilenet_v3_small(pretrained=True)
        self.model.eval()
        self.features = self.model.features.to(device)
        self.projector = nn.Linear(576, out_dim).to(device)  # MobileNetV3-Smallの最終特徴量次元数は576

        self.transform = T.Compose([
            T.Resize((224, 224)),
            T.ToTensor(),
            T.Normalize([0.485, 0.456, 0.406],
                        [0.229, 0.224, 0.225]),
        ])

    def encode(self, images, normalize: bool = True) -> torch.Tensor:
        """
        画像リストをエンコードしてベクトルを返す関数

        images: list[PIL.Image.Image] または 単一のPIL.Image.Image
        normalize: bool - 出力ベクトルを正規化するかどうか
        """
        single = False
        if not isinstance(images, (list, tuple)):
            images = [images]
            single = True

        tensors = []
        for img in images:
            pil_img = load_image(img)
            tensor = self.transform(pil_img) # (3, H, W)
            tensors.append(tensor)
        batch = torch.stack(tensors, dim=0).to(self.device)  # (B, 3, H, W)

        with torch.no_grad():
            feat = self.features(batch)  # (B, 576, 7, 7)
            feat = feat.mean(dim=[2, 3])  # (B, 576)

        proj = self.projector(feat)  # (B, out_dim)

        if normalize:
            proj = proj / (proj.norm(dim=1, keepdim=True) + 1e-8)

        if single:
            return proj # (1, out_dim)
        return proj  # (B, out_dim)

In [6]:
class MiniCLIP:
    """
    TextEncoder と ImageEncoder を共有 256次元空間に揃え、
    CLIP風の対照学習で projector 層のみを学習するクラス。
    """
    def __init__(self, text_encoder: TextEncoder, image_encoder: ImageEncoder, temperature: float = 0.07):
        self.text_encoder = text_encoder
        self.image_encoder = image_encoder
        self.temperature = temperature

        params = list(self.text_encoder.projector.parameters()) + \
                 list(self.image_encoder.projector.parameters())
        self.optimizer = AdamW(params, lr=1e-4)


    def compute_loss(self, img_vecs: torch.Tensor, txt_vecs: torch.Tensor) -> torch.Tensor:
        """
        CLIP風の対照学習の損失を計算する関数
        img_vecs: (B, D)
        txt_vecs: (B, D)
        """
        # 類似度行列[B, B]を計算
        sim_matrix = torch.matmul(img_vecs, txt_vecs.T)
        sim_matrix = sim_matrix / self.temperature

        labels = torch.arange(len(img_vecs), device=sim_matrix.device)

        loss_img2txt = F.cross_entropy(sim_matrix, labels)
        loss_txt2img = F.cross_entropy(sim_matrix.T, labels)

        loss = (loss_img2txt + loss_txt2img) / 2.0
        return loss


    def train(self, pairs, epochs: int = 10, batch_size: int = 32):
        """
        学習を実行する関数
        pairs: list of (PIL.Image.Image, str)
        """

        for epoch in range(epochs):
            self.optimizer.zero_grad()

            texts  = [p["text"]  for p in pairs]
            images = [p["image"] for p in pairs]

            txt_vecs = self.text_encoder.encode(texts, normalize=True)  # (B, D)
            img_vecs = self.image_encoder.encode(images, normalize=True)  # (B, D)

            loss = self.compute_loss(img_vecs, txt_vecs)
            loss.backward()
            self.optimizer.step()

            print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}")

        print("Training completed.")

In [7]:
class SimpleVectorDB:
    """
    非常にシンプルなベクトルストア。
    - items: List[{"vec": np.ndarray, "type": ..., "text": ..., "image": ...}]
    """
    def __init__(self):
        self.items = []

    def add(self, vec: np.ndarray, metadata: dict):
        self.items.append({
            "vec": vec.astype("float32"),
            **metadata
        })

    def build_from_pairs(self, text_encoder: TextEncoder, image_encoder: ImageEncoder, pairs):
        """
        pairs: List[{"image": ..., "text": ...}]
        """
        self.items = []
        for p in pairs:
            # text embedding
            t_vec = text_encoder.encode(p["text"]).cpu().detach().numpy()[0]
            self.add(t_vec, {
                "type": "text",
                "text": p["text"],
                "image": p["image"],
            })

            # image embedding
            i_vec = image_encoder.encode(p["image"]).cpu().detach().numpy()[0]
            self.add(i_vec, {
                "type": "image",
                "text": p["text"],
                "image": p["image"],
            })

    def search(self, query_vec: np.ndarray, top_k: int = 5, type_filter: str = None):
        """
        query_vec: np.ndarray [D]
        type_filter: "text" or "image" or None
        """
        results = []
        for item in self.items:
            if type_filter is not None and item["type"] != type_filter:
                continue
            sim = cosine_sim(query_vec, item["vec"])
            results.append((sim, item))

        results.sort(key=lambda x: x[0], reverse=True)
        return results[:top_k]

    def to_jsonable(self):
        """
        JSON保存用に numpy 配列を list に変換
        """
        json_items = []
        for item in self.items:
            j = dict(item)
            j["vec"] = item["vec"].tolist()
            json_items.append(j)
        return json_items

    @staticmethod
    def from_json(data):
        db = SimpleVectorDB()
        for item in data:
            vec = np.array(item["vec"], dtype="float32")
            meta = {k: v for k, v in item.items() if k != "vec"}
            db.add(vec, meta)
        return db



# Dataset

In [63]:
import kagglehub

class ArtBenchDatasetLoader:
    """
    ArtBench-10 の簡易データローダー。
    各フォルダ名をそのままテキスト説明として使う。
    データはkagglehubからダウンロード

    limit_per_class: 1クラスあたりの最大データ数。
    shuffle: データをシャッフルするかどうか。
    """
    def __init__(self, root_dir, limit_per_class=200, shuffle=True):
        self.root = root_dir
        self.csv_path = os.path.join(root_dir, "ArtBench-10.csv")
        self.limit = limit_per_class
        self.shuffle = shuffle

        # CSV の構造に基づく正しい mapping
        df = pd.read_csv(self.csv_path)
        if "label" not in df.columns:
            raise RuntimeError(f"'label' 列が存在しません: {df.columns}")

        self.label_list = df["label"].tolist()


    def _load_batch(self, file_path):
        batch = unpickle(file_path)
        data = batch[b"data"]              # (N, 3072)
        labels = batch[b"labels"]          # list of ints
        data = data.reshape(-1, 3, 32, 32) # CIFAR shape
        return data, labels


    def _to_pil(self, arr3x32x32):
        """numpy array (3,32,32) → PIL.Image"""
        # CIFAR形式 (C,H,W) → (H,W,C)
        img = np.transpose(arr3x32x32, (1, 2, 0))
        return Image.fromarray(img.astype(np.uint8))


    def load(self):
        """
        画像 + テキスト (style の説明) のペアを返す。
        """
        batch_files = [
            f for f in os.listdir(self.root)
            if f.startswith("data_batch") or f == "test_batch"
        ]
        batch_files.sort()

        pairs = []

        for bf in batch_files:
            data, labels = self._load_batch(os.path.join(self.root, bf))

            per_class_count = {}

            for i, lid in enumerate(labels):

                per_class_count.setdefault(lid, 0)
                if per_class_count[lid] >= self.limit:
                    continue

                style_name = self.label_list[i]  # CSVの 'label' は cifar_index で対応

                img_pil = self._to_pil(data[i])
                text = f"{style_name} の絵画"

                pairs.append({
                    "image": img_pil,
                    "text": text
                })

                per_class_count[lid] += 1

        if self.shuffle:
            np.random.shuffle(pairs)

        print(f"Loaded {len(pairs)} image-text pairs from ArtBench10.")
        return pairs


In [9]:
class DogDemoDatasetLoader:
    """
    とりあえず dog.ceo のデモデータを返す Loader。
    将来、ArtBenchLoader / MemotionLoader / CustomLoader を同じIFで追加できる。
    """
    def load(self):
        pairs = [
            {
                "image": "https://images.dog.ceo/breeds/chow/n02112137_16777.jpg",
                "text": "白いフワフワの犬"
            },
            {
                "image": "https://images.dog.ceo/breeds/pitbull/20190710_143021.jpg",
                "text": "黒と白の犬が緑の草原の中にいる"
            },
            {
                "image": "https://images.dog.ceo/breeds/poodle-toy/n02113624_9550.jpg",
                "text": "ふわふわの子犬の写真"
            },
            {
                "image": "https://images.dog.ceo/breeds/eskimo/n02109961_8185.jpg",
                "text": "白い犬が石畳の上に座っている"
            }
        ]
        return pairs

loader = DogDemoDatasetLoader()
image_text_pairs = loader.load()
len(image_text_pairs), image_text_pairs[0]

(4,
 {'image': 'https://images.dog.ceo/breeds/chow/n02112137_16777.jpg',
  'text': '白いフワフワの犬'})

# MiniClip作って学習

In [ ]:
# DownloadDataset
!mkdir -p ./artbench
!kaggle datasets download alexanderliao/artbench10 -p ./artbench --unzip

In [61]:
loader = ArtBenchDatasetLoader("./artbench", limit_per_class=200)
# (image, text) ペアを取得
pairs = loader.load()
print(len(pairs))

# MiniCLIP に投入
# mini_clip.train(pairs, epochs=10)

RuntimeError: ArtBench-10.csv の列構造が想定と違います

In [62]:
import pandas as pd

df = pd.read_csv("/content/artbench/ArtBench-10.csv")  # ←パスは合わせて
print(df.head())
print(df.columns)


                                            name              artist  \
0      frank-omeara_towards-night-and-winter.jpg        frank-omeara   
1                 goldstein-grigoriy_morning.jpg  goldstein-grigoriy   
2                 georges-lemmen_man-reading.jpg      georges-lemmen   
3       theodor-aman_port-of-constantza-1882.jpg        theodor-aman   
4  niccolo-cannicci_il-passo-della-futa-1914.jpg    niccolo-cannicci   

                                                 url  is_public_domain  \
0  https://uploads5.wikiart.org/00316/images/fran...              True   
1  https://uploads5.wikiart.org/images/grigoriy-g...              True   
2  https://uploads6.wikiart.org/images/georges-le...              True   
3  https://uploads6.wikiart.org/images/theodor-am...              True   
4  https://uploads3.wikiart.org/images/niccolo-ca...              True   

   length  width          label  split  cifar_index  
0     800    657  impressionism  train        43186  
1     521    4

# VectorDB構築＆検索＆可視化

In [ ]:
# VectorDB構築
vecdb = SimpleVectorDB()
vecdb.build_from_pairs(text_encoder, image_encoder, image_text_pairs)
print(f"VectorDB size: {len(vecdb.items)} items")

# テキストでクエリ
query_text = "白い犬"
query_vec = text_encoder.encode(query_text).cpu().detach().numpy()[0]

results = vecdb.search(query_vec, top_k=5)
for sim, item in results:
    print(f"sim={sim:.3f} | type={item['type']} | text={item['text']} | image={item['image']}")
    img = load_image(item["image"])
    img_small = img.resize((150, 100))
    display(img_small)

# UMAP 可視化

In [ ]:
embs = np.stack([it["vec"] for it in vecdb.items], axis=0)
labels = [it["type"] for it in vecdb.items]
texts = [it["text"] for it in vecdb.items]

# plot_umap_matplotlib(embs, labels, texts, title="Mini-CLIP: Dog Demo (Matplotlib)")
plot_umap_plotly(embs, labels, texts, title="Mini-CLIP: Dog Demo (Plotly)")

# Export image

In [ ]:
os.makedirs("artifacts", exist_ok=True)

# projector 保存
torch.save(text_encoder.projector.state_dict(), "artifacts/text_projector.pt")
torch.save(image_encoder.projector.state_dict(), "artifacts/image_projector.pt")

# VectorDB 保存
with open("artifacts/vector_db.json", "w", encoding="utf-8") as f:
    json.dump(vecdb.to_jsonable(), f, ensure_ascii=False, indent=2)

print("Saved to ./artifacts/")

#### 日本語表示対応

In [ ]:
# # 日本語フォントをダウンロードする。
# !apt-get -y install fonts-ipafont-gothic
# import matplotlib.pyplot as plt
# plt.rcParams['font.family'] = 'IPAGothic'

# import shutil
# import os
# # フォントキャッシュを削除
# font_cache_path = os.path.expanduser("~/.cache/matplotlib")
# if os.path.exists(font_cache_path):
#     shutil.rmtree(font_cache_path)